In [2]:
# I chose the Exponential Smoothing (Holt-Winters) model

from __future__ import annotations
from pathlib import Path
import pickle
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing


def _prep_train_series(train_df: pd.DataFrame) -> pd.Series:

    df = train_df.copy()
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    df = df.sort_values("Timestamp").set_index("Timestamp")

    y = pd.to_numeric(df["trips"], errors="coerce")
    # Force hourly frequency to keep Holt-Winters happy and reproducible
    y = y.asfreq("H")

    # Fill any occasional missing values to maintain a clean hourly series
    if y.isna().any():
        y = y.interpolate(limit_direction="both")

    return y


def _prep_test_index(test_df: pd.DataFrame) -> pd.DatetimeIndex:

    idx = pd.to_datetime(test_df["Timestamp"])
    try:
        # If timestamps are hourly, this preserves the hourly freq metadata
        idx = pd.DatetimeIndex(idx).asfreq("H")
    except Exception:
        idx = pd.DatetimeIndex(idx)
    return idx


def model1(train_df: pd.DataFrame):


    y = _prep_train_series(train_df)

    model = ExponentialSmoothing(
        y,
        trend="add",
        seasonal="add",
        seasonal_periods=24,              # daily seasonality for hourly data
        initialization_method="estimated" # robust init across statsmodels versions
    )

    fitted = model.fit(optimized=True)
    return fitted


def predict(fitted_model, test_df: pd.DataFrame) -> pd.Series:

    h = len(test_df)
    fc = fitted_model.forecast(steps=h)
    idx = _prep_test_index(test_df)
    return pd.Series(fc.values, index=idx, name="pred")


def save_model(fitted_model, path: str | Path = "model1.pkl") -> Path:

    p = Path(path)
    with p.open("wb") as f:
        pickle.dump(fitted_model, f)
    return p


def load_model(path: str | Path = "model1.pkl"):

    p = Path(path)
    with p.open("rb") as f:
        return pickle.load(f)
